In [ ]:
## Adding the .simplify_full() command may give cleaner functions to look at, but drastically increases the runtime (from 0.5 sec to more than 4 min), and, most importantly, causes completely unstable computations that ruin the precision (for unknown reasons).

prec = 200
R = RealBallField(prec)
var('x')

W = (-x/2 + sqrt(1/27 + x^2/4))^(1/3) - (x/2 + sqrt(1/27 + x^2/4))^(1/3)
W_prime = diff(W, x)
a = 1 + W/x + W_prime
b = (3*x)/2 + W

# Number of derivatives to compute
n = 7

a_derivs = [a]
b_derivs = [b]

for k in range(1, n+1):
    a_deriv_k = diff(a, x, k)
    b_deriv_k = diff(b, x, k)
    a_derivs.append(a_deriv_k)
    b_derivs.append(b_deriv_k)

# Convert symbolic functions to numerical ones evaluable on RealBallField elements
a_funcs = [fast_callable(expr, vars=[x], domain=R) for expr in a_derivs]
b_funcs = [fast_callable(expr, vars=[x], domain=R) for expr in b_derivs]

In [10]:
x0 = R(1)

print("Evaluations of the derivatives at x = 1:")
for k in range(n+1):
    a_val = a_funcs[k](x0)
    b_val = b_funcs[k](x0)
    print(f"d^{k}a/dx^{k}({x0}) = {a_val}")
    print(f"d^{k}b/dx^{k}({x0}) = {b_val}")

Evaluations of the derivatives at x = 1:
d^0a/dx^0(1.0000000000000000000000000000000000000000000000000000000000000) = [-0.0995657917542381049909592561213408039935809253791502361938 +/- 5.02e-59]
d^0b/dx^0(1.0000000000000000000000000000000000000000000000000000000000000) = [0.8176721961719806726305162602889517431088114181020014221963 +/- 4.47e-59]
d^1a/dx^1(1.0000000000000000000000000000000000000000000000000000000000000) = [0.562458921144505356640633633750586484017539061910301727256 +/- 3.37e-58]
d^1b/dx^1(1.0000000000000000000000000000000000000000000000000000000000000) = [1.0827620120737812223785244835897074528976076565188483416099 +/- 2.81e-59]
d^2a/dx^2(1.0000000000000000000000000000000000000000000000000000000000000) = [-0.686784262510245927938463483009399083480164248805471277239 +/- 4.90e-58]
d^2b/dx^2(1.0000000000000000000000000000000000000000000000000000000000000) = [0.2973691052427048068926254104498307742287428234934548078426 +/- 6.46e-59]
d^3a/dx^3(1.00000000000000000000000000000